# Config

In [ ]:
# import src modules
import sys
import os

sys.path.append(os.path.abspath("../src"))

# 1. Generate Test Dataset

In [ ]:
from test_dataset_generation import generate_test_dataset

In [ ]:
generate_test_dataset()

# 2. RAG System